In [3]:
# 05/22

* competition site :https://www.kaggle.com/c/competitive-data-science-predict-future-sales
* Reference code1 :https://www.kaggle.com/code/zuqilaksamana/future-sales-forecasting-ar-arima-prophet#ARIMA
* Reference code2:https://www.kaggle.com/code/avikumart/sales-prediction-using-sklearn-statsmodels-api/notebook#Model-1
* Reference code3 https://www.kaggle.com/code/joanponsv/future-sales-prediction-0p98


* The version is the promotion of privious version .
* The key action is strengthen extraction of feature and creation of new-feature.

In [4]:
import numpy as np 
import pandas as pd
import datetime 
from sklearn.metrics import mean_squared_error
from numpy import sqrt
import seaborn as sns
from itertools import product
from xgboost import XGBRegressor

import matplotlib.pyplot as plt 
%matplotlib inline

# from statsmodels.tsa.ar_model import AutoReg
# from statsmodels.tsa.arima.model import ARIMA
# from fbprophet import Prophet

# from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
# from statsmodels.tsa.stattools import kpss

In [5]:
titem = pd.read_csv("c:/csv/feature_sale/items.csv")
titemcat = pd.read_csv("c:/csv/feature_sale/item_categories.csv")
tsale = pd.read_csv("c:/csv/feature_sale/sales_train.csv")
tshop = pd.read_csv("c:/csv/feature_sale/shops.csv")
ttest= pd.read_csv("c:/csv/feature_sale/test.csv")
tsamsub= pd.read_csv("c:/csv/feature_sale/sample_submission.csv")

In [6]:
tsale.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


In [7]:
tsale.date

0          02.01.2013
1          03.01.2013
2          05.01.2013
3          06.01.2013
4          15.01.2013
              ...    
2935844    10.10.2015
2935845    09.10.2015
2935846    14.10.2015
2935847    22.10.2015
2935848    03.10.2015
Name: date, Length: 2935849, dtype: object

# 1 Overview and Hypothesis

** Data Description:
* ID - an Id that represents a (Shop, Item) tuple within the test set
* shop_id - unique identifier of a shop
* item_id - unique identifier of a product
* item_category_id - unique identifier of item category
* item_cnt_day - number of products sold. You are predicting a monthly amount of this measure
* item_price - current price of an item
* date - date in format dd/mm/yyyy
* date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
* item_name - name of item
* shop_name - name of shop
* item_category_name - name of item category

**Hypothesis :
* Sale may related to locatoin of shop .
* Sale may related to subclass/classification of item.
* Sale may related to season/week。 

## 1.1 observation target 

In [8]:
display(ttest)

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
...,...,...,...
214195,214195,45,18454
214196,214196,45,16188
214197,214197,45,15757
214198,214198,45,19648


* The target is estimation values of monthly sale of some item,\
it saying, base on shopID and itemID to predicate its monthly sale.

In [9]:
display(titem)
display(titemcat)

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40
...,...,...,...
22165,"Ядерный титбит 2 [PC, Цифровая версия]",22165,31
22166,Язык запросов 1С:Предприятия [Цифровая версия],22166,54
22167,Язык запросов 1С:Предприятия 8 (+CD). Хрустале...,22167,49
22168,Яйцо для Little Inu,22168,62


,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4
...,...,...
79,Служебные,79
80,Служебные - Билеты,80
81,Чистые носители (шпиль),81
82,Чистые носители (штучные),82


 ** Hyothesis  1：
 * the month sale is relate to the 
 * item_name of titem should be separate by subitemname so that can be create new feature.

* the range of date_block_num  is (0,33),which represent month.

In [10]:
tsale["item_price"].describe()

count    2.935849e+06
mean     8.908532e+02
std      1.729800e+03
min     -1.000000e+00
25%      2.490000e+02
50%      3.990000e+02
75%      9.990000e+02
max      3.079800e+05
Name: item_price, dtype: float64

In [11]:
tsale.isnull().sum()

date              0
date_block_num    0
shop_id           0
item_id           0
item_price        0
item_cnt_day      0
dtype: int64

In [12]:
display(tshop)
display(tsamsub)

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4
5,"Вологда ТРЦ ""Мармелад""",5
6,"Воронеж (Плехановская, 13)",6
7,"Воронеж ТРЦ ""Максимир""",7
8,"Воронеж ТРЦ Сити-Парк ""Град""",8
9,Выездная Торговля,9


,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5
...,...,...
214195,214195,0.5
214196,214196,0.5
214197,214197,0.5
214198,214198,0.5


** Hyothesis 2:
* the sale is related to shop location 


# 2 Data preprocessing 

## 2.1 for item_categories 

* First element of item_category_name indicate cityname  of shop located in.

In [13]:
titemcat = pd.read_csv("c:/csv/feature_sale/item_categories.csv")

In [14]:
titemcat

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4
...,...,...
79,Служебные,79
80,Служебные - Билеты,80
81,Чистые носители (шпиль),81
82,Чистые носители (штучные),82


In [15]:
titemcat.shape

(84, 2)

In [16]:
titemcat.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [17]:
titemcat_sub=titemcat["item_category_name"].str.split('-',expand=True)
print(titemcat_sub.shape)
titemcat_sub.head()

(84, 3)


,0,1,2
0,PC,Гарнитуры/Наушники,None
1,Аксессуары,PS2,None
2,Аксессуары,PS3,None
3,Аксессуары,PS4,None
4,Аксессуары,PSP,None


In [18]:
titemcat.shape

(84, 2)

In [19]:
titemcat_ext=pd.merge(titemcat,titemcat_sub,how="left",left_index=True,right_index=True)

In [20]:
print(titemcat_ext.shape)
titemcat_ext.head()

(84, 5)


,item_category_name,item_category_id,0,1,2
0,PC - Гарнитуры/Наушники,0,PC,Гарнитуры/Наушники,None
1,Аксессуары - PS2,1,Аксессуары,PS2,None
2,Аксессуары - PS3,2,Аксессуары,PS3,None
3,Аксессуары - PS4,3,Аксессуары,PS4,None
4,Аксессуары - PSP,4,Аксессуары,PSP,None


In [21]:
titemcat.rename(columns={0:"item_subclass1",1:"item_subclass2"},inplace=True)
titemcat

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4
...,...,...
79,Служебные,79
80,Служебные - Билеты,80
81,Чистые носители (шпиль),81
82,Чистые носители (штучные),82


In [22]:
titemcat_ext.drop(labels=2,axis=1,inplace=True)
titemcat_ext

,item_category_name,item_category_id,0,1
0,PC - Гарнитуры/Наушники,0,PC,Гарнитуры/Наушники
1,Аксессуары - PS2,1,Аксессуары,PS2
2,Аксессуары - PS3,2,Аксессуары,PS3
3,Аксессуары - PS4,3,Аксессуары,PS4
4,Аксессуары - PSP,4,Аксессуары,PSP
...,...,...,...,...
79,Служебные,79,Служебные,None
80,Служебные - Билеты,80,Служебные,Билеты
81,Чистые носители (шпиль),81,Чистые носители (шпиль),None
82,Чистые носители (штучные),82,Чистые носители (штучные),None


## 2.2 for shops 

In [23]:
tshop = pd.read_csv("c:/csv/feature_sale/shops.csv")

In [24]:
print(tshop.shape)
tshop.head()

(60, 2)


,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


* First element of shop_name indicate cityname of shop.

In [25]:
tshop_sub1=tshop["shop_name"].str.split(" ",expand=True)[0]
print(tshop_sub1.shape)
tshop_sub1.head()

(60,)


0     !Якутск
1     !Якутск
2      Адыгея
3    Балашиха
4    Волжский
Name: 0, dtype: object

In [26]:
tshop_ext=pd.merge(tshop,tshop_sub1,how="left",left_index=True,right_index=True)
print(tshop_ext.shape)
tshop_ext.head()

(60, 3)


,shop_name,shop_id,0
0,"!Якутск Орджоникидзе, 56 фран",0,!Якутск
1,"!Якутск ТЦ ""Центральный"" фран",1,!Якутск
2,"Адыгея ТЦ ""Мега""",2,Адыгея
3,"Балашиха ТРК ""Октябрь-Киномир""",3,Балашиха
4,"Волжский ТЦ ""Волга Молл""",4,Волжский


In [27]:
tshop_ext.rename(columns={0:"city"},inplace=True)
tshop_ext.head()

,shop_name,shop_id,city
0,"!Якутск Орджоникидзе, 56 фран",0,!Якутск
1,"!Якутск ТЦ ""Центральный"" фран",1,!Якутск
2,"Адыгея ТЦ ""Мега""",2,Адыгея
3,"Балашиха ТРК ""Октябрь-Киномир""",3,Балашиха
4,"Волжский ТЦ ""Волга Молл""",4,Волжский


# 2.3 for sales_train

* processing outlier and abnormal values

### 2.3.1  outliers droppoing 

In [28]:
Q1 = np.percentile(tsale['item_price'], 25.0)
Q3 = np.percentile(tsale['item_price'], 75.0)

IQR = Q3 - Q1

df_sub1 = tsale[tsale['item_price'] > Q3 + 1.5*IQR]
df_sub2 = tsale[tsale['item_price'] < Q1 - 1.5*IQR]

tsale.drop(df_sub1.index, inplace=True)

tsale.shape

(2676907, 6)

### 2.3.2 drop abnormal value  and duplication 

In [29]:
# drop samples which contains negative value of item_price
df_sub3=tsale[tsale['item_price']<0]
tsale.drop(df_sub3.index, inplace=True)
tsale.shape

(2676906, 6)

In [30]:
df_sub4=tsale[tsale['item_cnt_day']<0]
tsale.drop(df_sub4.index, inplace=True)
tsale.shape

(2670937, 6)

In [31]:
tsale.drop_duplicates(keep='first',inplace=True)

In [32]:
tsale.shape

(2670931, 6)

In [33]:
tsale.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
5,10.01.2013,0,25,2564,349.00,1.0


### 2.3.3 create new feature of item_cnt_month

In [34]:
# target of item_cnt_month aim to predicate monthly sale of ID(combination of itemID and shopID )
item_cnt_month=tsale.groupby(["date_block_num","shop_id","item_id"],as_index=False).agg(item_cnt_month=("item_cnt_day", "sum"))
print(item_cnt_month.shape)
item_cnt_month

(1499844, 4)


,date_block_num,shop_id,item_id,item_cnt_month
0,0,0,32,6.0
1,0,0,33,3.0
2,0,0,35,1.0
3,0,0,43,1.0
4,0,0,51,2.0
...,...,...,...,...
1499839,33,59,22087,6.0
1499840,33,59,22088,2.0
1499841,33,59,22091,1.0
1499842,33,59,22100,1.0


In [35]:
tsale_ext1= pd.merge(tsale,item_cnt_month,how='left',on=["date_block_num","shop_id","item_id"])

In [36]:
display(tsale_ext1)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_cnt_month
0,02.01.2013,0,59,22154,999.00,1.0,1.0
1,03.01.2013,0,25,2552,899.00,1.0,1.0
2,06.01.2013,0,25,2554,1709.05,1.0,1.0
3,15.01.2013,0,25,2555,1099.00,1.0,1.0
4,10.01.2013,0,25,2564,349.00,1.0,1.0
...,...,...,...,...,...,...,...
2670926,10.10.2015,33,25,7409,299.00,1.0,3.0
2670927,09.10.2015,33,25,7460,299.00,1.0,3.0
2670928,14.10.2015,33,25,7459,349.00,1.0,1.0
2670929,22.10.2015,33,25,7440,299.00,1.0,1.0


### 2.3.4 identify the closed shop in last month

** Assumption:
* If a shop Not sell any items in last month ,it is considerated to be closed.
* the date_block_num imply month paramater and its range in (0-33)

In [37]:
lastmonth=33
closeshopID=[]
for shopid in tsale["shop_id"].unique():
    monthid=tsale[tsale["shop_id"]==shopid]["date_block_num"]
    if np.max(monthid)<33:
        closeshopID.append(shopid)
print("closeshopID",closeshopID)

closeshopID [23, 27, 29, 0, 1, 8, 13, 30, 32, 54, 43, 51, 17, 40, 33, 11]


## 2.4 create feature Matrix by Cartesian product

### 2.4.1 obtain the set difference about tsale_ext1.date_block_nam  and closemonth

By *** np.setdiff1d ***in order to reduce invalid number shop to put into production .


In [ ]:
featCols=["date_block_num","shopID","itemID","date_block_num"]
firstMonth = 20 
shop_Id=list(np.setdiff1d(tsale["shop_id"].unique(),closeshopID))
dateblocknum=tsale[(tsale["date_block_num"]>=0) &(tsale["date_block_num"]<12)]["date_block_num"]
featMatr=[]
for shopid in shop_Id:
    for date_block_num in dateblocknum:
        itemId=tsale.loc[tsale["shop_id"]==shopid]["item_id"]
        featMatr.append(np.array(list(product(*[[date_block_num],[shopid], itemId])),dtype='int32'))
featureMatrix = pd.DataFrame(np.vstack(featMatr), columns = featCols,dtype=np.int32) 

In [ ]:
dateblocknum=tsale[tsale["date_block_num"]%2==0]["date_block_num"]
dateblocknum

In [ ]:
************************************************************************************

# Cancel 3.2  create new column

In [ ]:
ttrain=tsale.copy()

In [ ]:
# cocluating mean of item_price by each date_block_num
price = round(np.array(ttrain.groupby('date_block_num')['item_price'].mean()).mean(),2)
print(price)

In [ ]:
dict(round(ttrain.groupby('date_block_num')['item_price'].mean(),4))

In [ ]:
replace_dict = dict(round(ttrain.groupby('date_block_num')['item_price'].mean(),2))

In [ ]:
ttrain['date_block_num'] = ttrain['date_block_num'].replace(replace_dict)
ttrain.rename(columns = {'date_block_num':'mean_price_by_month'}, inplace=True)

In [ ]:
ttrain

In [ ]:
comb_data=pd.concat([ttrain,ttest])

In [ ]:
comb_data

In [ ]:
comb_data['mean_price_by_column'] = comb_data['mean_price_by_month'].fillna(value=price)
comb_data['item_cnt_day'] = comb_data['item_cnt_day'].fillna(value=0)

test_df = comb_data[comb_data['item_cnt_day'] == 0]
train_df = comb_data[comb_data['item_cnt_day'] != 0]

In [ ]:
train_df.drop(["date","item_price","ID","mean_price_by_column"],axis=1,inplace=True)

In [ ]:
test_df.drop(["date","item_price","ID","mean_price_by_column"],axis=1,inplace=True)

In [ ]:
train_df

In [ ]:
test_df

In [ ]:
X_train = train_df.loc[:,['mean_price_by_month','shop_id','item_id']]
y_train = train_df.loc[:,'item_cnt_day']
X_test= test_df.loc[:,['mean_price_by_month','shop_id','item_id']]

# 3.3 XGboost

In [ ]:
model = XGBRegressor(n_estimators=50,max_depth=3,
                      learning_rate = 0.01)

model.fit(X_train, y_train)

prey = model.predict(X_test)

In [ ]:
prey

In [ ]:
prey.shape

In [ ]:
ttest.shape

In [ ]:
result = pd.DataFrame(data= {'ID':np.array(ttest['ID']),'item_cnt_month':prey})

In [ ]:
result

In [ ]:
result.to_csv('c:/csv/feautresale_score0516.csv', index=False) 

In [ ]:
***********************************************************************************

In [ ]:
**************************************************************************************************************************

# 2   TimeSerise Model

## 3.1  tranform day_sale to month_sale
